В данном примере происходит обучение нейросети LSTM типа на последовательноси синуса.

In [ ]:
import tensorflow as tf
import time
import matplotlib
from matplotlib import pyplot
%run lstm.ipynb

## Гиперпараметры

In [ ]:
num_epochs = 25
num_repeat = 2
input_dim = 5
hidden_dim = 5
output_dim = input_dim
learn_rate = 0.5
pruning_iters = 20
train_to_test_ratio = 0.7
pruning_strength = 0.95 # <= меньше сильнее
optimizer = 'adagrad'

data_sequence = 'ascending-sine' # <= sine, ascending-sine
do_normalize = True
do_pruning = False # <= производить SVD разложение матриц весов

## Подготовка данных

In [ ]:
def split_data(data, ratio):
    num_samples = int(len(data) * ratio)
    return data[:num_samples], data[num_samples:]

In [ ]:
def shift(seq, seq_size, step=1):
    x, y = [], []
    for i in np.arange(0, len(seq) - seq_size - step + 1, step):
        xc = seq[i:i+seq_size]
        yc = seq[i+step:i+seq_size+step]
        x.append(xc)
        y.append(yc)
    return x, y

In [ ]:
x = np.linspace(0., 20. * np.pi, 200)

# Здесь работает переключение способа генерации данных для обработки.
if data_sequence == 'sine':
    seq = np.sin(x)
elif data_sequence == 'ascending-sine':
    seq =  np.sin(x) + 0.1 * x + 0.5

if do_normalize == True:
    seq_max = np.amax(seq)
    seq /= seq_max
    
seq_train, seq_test = split_data(seq, train_to_test_ratio)
x_train, y_train = shift(seq_train, input_dim, step=1)

matplotlib.pyplot.plot(np.arange(0., len(seq_train), 1.), seq_train)
matplotlib.pyplot.plot(np.arange(len(seq_train), len(seq_train) + len(seq_test), 1.), seq_test)
#matplotlib.pyplot.plot(np.linspace(len(x_train), len(x_train) + len(x_test), len(x_test)), x_test)

## Обучение

In [ ]:
def train(net, x_train, y_train):
    
    begin = time.time()

    for e in range(num_epochs):
        net.clear_state()
        for x, y in zip(x_train, y_train):
            net.train([x], [y], repeat=num_repeat)

    return time.time() - begin

In [ ]:
# Функция производит предсказание последовательности на указанное количество шагов length.
# За актуальностью внутреннего состояния LSTM до и после вызова следит вызывающий методв.
def predict(net, start, length):
    
    seq_prev = [start]
    seq_points = []

    for i in range(length):
        seq_prev = net.test(seq_prev)
        seq_points.append(seq_prev[:,-1])
        
    return seq_points

In [ ]:
# Сзедняя абсолютная ошибка.
def mae(a, b):
    length = len(a)
    assert length == len(b)
    return np.sum(np.abs(a - b)) / length

In [ ]:
# Средняя квадратическая ошибка.
def mse(a, b):
    length = len(a)
    assert length == len(b)
    return np.sum(np.square(a - b)) / length

In [ ]:
sess = tf.Session()
net = lstm_cell(sess, input_dim, hidden_dim, output_dim, learn_rate, optimizer)

In [ ]:
# Различные интересные зависимости.
c_mae = []
c_mse = []
c_norm = []
c_train_time = []

In [ ]:
for i in range(pruning_iters):
    
    train_time = train(net, x_train, y_train)
    net.save_all()
    
    #seq_points = predict(net, x_train[-1], len(seq_test))
    seq_points = predict(net, y_train[-1], len(seq_test))
    net.restore_all()
    
    if do_pruning == True:
        norm, _ = net.svd_compress(pruning_strength, pruning_strength)
        c_norm.append(norm)
        
    c_train_time.append(train_time)
    c_mae.append(mae(seq_points, seq_test))
    c_mse.append(mse(seq_points, seq_test))
    
    print(i, 'train_time', c_train_time[-1], 'norm', c_norm[-1] if c_norm else 0., 'mae', c_mae[-1], 'mse', c_mse[-1])

    if i > 2 and c_mae[-1] >= c_mae[-2]:
        print('Насыщение')
        break

    pass

# Так как на выходе на последней итерации производится SVD разложение, то нужно восстановить состояние LSTM.
net.restore_all()

## Предсказание

In [ ]:
net.restore_all()
seq_points = predict(net, y_train[-1], len(seq_test))

# Вывод графиков + статистика SVD
if False:
    
    fig, ax = matplotlib.pyplot.subplots(2, 3, figsize=(10, 5))

    ax[0,0].set_title('Обучение, сек.')
    ax[0,0].plot(c_train_time)

    ax[0,1].set_title('Норма SVD')
    ax[0,1].plot(c_norm)

    ax[1,0].set_title('MAE')
    ax[1,0].plot(c_mae)

    ax[1,1].set_title('MSE')
    ax[1,1].plot(c_mse)

    ax[0,2].set_title('Предсказание')
    ax[0,2].plot(seq_test, 'orange')
    ax[0,2].plot(seq_points, '--')

    # https://matplotlib.org/3.1.1/gallery/subplots_axes_and_figures/gridspec_and_subplots.html
    ax[1,2].remove()
    #ax[0,2].set_position([1.-0.3, 0.5-0.15, 0.3, 0.3])

# Графики без статистики SVD
elif False:
    
    fig, ax = matplotlib.pyplot.subplots(2, 2, figsize=(10, 5))

    ax[0,0].set_title('MAE')
    ax[0,0].plot(c_mae)

    ax[0,1].set_title('MSE')
    ax[0,1].plot(c_mse)
    
    ax[1,0].set_title('Обучение, сек.')
    ax[1,0].plot(c_train_time)

    ax[1,1].set_title('Предсказание')
    ax[1,1].plot(seq_test, 'orange')
    ax[1,1].plot(seq_points, '--')

# Только предсказание.
elif True:
    
    matplotlib.pyplot.plot(seq_test, 'orange')
    matplotlib.pyplot.plot(seq_points, '--')

# Отображаем статистику обучения.
print('optimizer', optimizer)
print('learn_rate', learn_rate)
print('train_time', np.sum(c_train_time))
print('pruning_iters', pruning_iters)
print('num_epochs', num_epochs)
print('num_repeat', num_repeat)
print('input_dim', input_dim)
print('hidden_dim', hidden_dim)
print('output_dim', output_dim)
print('do_normalize', do_normalize)
print('do_pruning', do_pruning)
print('MAE', c_mae[-1])
print('MSE', c_mse[-1])